# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#pull csv from weatherpy
path = "../output_data/cities.csv"
city_data = pd.read_csv(path)

city_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,jaguariaiva,49,BR,1580246518,50,86.00,-24.25,-49.71,3.00
1,1,cape town,75,ZA,1580246283,88,71.01,-33.93,18.42,9.17
2,2,port elizabeth,75,ZA,1580246518,88,64.40,-33.92,25.57,14.99
3,3,punta arenas,40,CL,1580246290,53,53.60,-53.15,-70.92,33.33
4,4,poum,100,NC,1580246466,79,79.50,-20.23,164.02,15.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_data[["Lat", "Lng"]]


In [4]:
locations = city_data[["Lat", "Lng"]]
weights = city_data['Humidity']
fig = gmaps.figure(center=(20.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_name = []


for index, row in city_data.iterrows():
    if row["Cloudiness"] == 0:
        if row ["Wind Speed"] < 10:
            if row["Temp"] >70 and row["Temp"]<90:
                city_name.append(row)

city_weather_df = pd.DataFrame(city_name)

city_weather_df.to_csv("city_perfect_weather.csv")
city_weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
57,57,basoko,0,CD,1580246523,43,75.27,1.24,23.62,0.83
107,107,yuli,0,NG,1580246528,15,74.73,9.70,10.27,7.27
109,109,cintalapa,0,MX,1580246528,32,84.76,16.70,-93.73,9.19
122,122,lujan,0,AR,1580246529,48,88.00,-34.57,-59.10,9.17
160,160,chuy,0,UY,1580246533,53,76.53,-33.70,-53.46,9.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input="

input = "Hotel"
radius = "5000"


hotel_df = city_weather_df
hotel_df["Hotel Name"] = ''
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
57,57,basoko,0,CD,1580246523,43,75.27,1.24,23.62,0.83,
107,107,yuli,0,NG,1580246528,15,74.73,9.70,10.27,7.27,
109,109,cintalapa,0,MX,1580246528,32,84.76,16.70,-93.73,9.19,
122,122,lujan,0,AR,1580246529,48,88.00,-34.57,-59.10,9.17,
160,160,chuy,0,UY,1580246533,53,76.53,-33.70,-53.46,9.42,


In [9]:
hotel_df.count()

Unnamed: 0    16
City          16
Cloudiness    16
Country       16
Date          16
Humidity      16
Temp          16
Lat           16
Lng           16
Wind Speed    16
Hotel Name    16
dtype: int64

In [13]:
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    target_url = (f"{base_url}{input}&key={g_key}&inputtype=textquery&fields=name,formatted_address&locationbias=circle:{radius}@{latitude},{longitude}")
    
    response = requests.get(target_url).json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = response["candidates"][0]["name"]

    except:
        print(f"There are no hotels here...")
        pass

hotel_df.to_csv('hotel_df.csv') 
hotel_df.head(20)


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
57,57,basoko,0,CD,1580246523,43,75.27,1.24,23.62,0.83,Flat Hotel La Benediction
107,107,yuli,0,NG,1580246528,15,74.73,9.70,10.27,7.27,Atlanta Marriott Marquis
109,109,cintalapa,0,MX,1580246528,32,84.76,16.70,-93.73,9.19,Hotel Marlene
122,122,lujan,0,AR,1580246529,48,88.00,-34.57,-59.10,9.17,Hilton Pilar
160,160,chuy,0,UY,1580246533,53,76.53,-33.70,-53.46,9.42,Pousada Quebra Mar
168,168,east london,0,ZA,1580246523,58,70.75,-33.02,27.91,8.23,Garden Court East London
196,196,tomatlan,0,MX,1580246536,35,83.48,19.93,-105.25,8.14,Hotel Hacienda Vieja
233,233,rocha,0,UY,1580246540,20,84.79,-34.48,-54.33,7.83,Posada Anacahuita
246,246,alappuzha,0,IN,1580246541,88,75.00,9.49,76.33,5.01,Chaandhni Lake View
273,273,avarua,0,CK,1580246544,88,82.40,-21.21,-159.78,8.05,The Edgewater Resort and Spa


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer on top of the heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))